In [1]:
import pandas as pd
import numpy as np
import json
import os
import math
import sys
import scipy.stats as st
from datetime import datetime
import re
import time

run ipip.py

In [2]:
# 转时区
to_ns = 8*60*60*1000*1000*1000
to_ms = 8*60*60*1000
#to_s = 8*60*60
to_s = 0

In [5]:
#所有“闪电借款”appid
jk_appid=['jkwx', 'jkand', 'jkand2', 'jkandroid', 'jkios', 'yzand1', 'jkiosspeed', 'jkiossolar','jkioslunar',
         'jkmjand_1','jkmjios_1','jkiosxjjsd','jkiosjqb','jkiosxysjd','jkiosjqd','jkiosxejsd','jkiossjdjk',
         'jkiosxjjk','jkiospphd','jkiosjyq','jkiosxedk','jkandjqb','jkandxedk','jkandpphd']
jk_appid_and = [ 'jkand', 'jkand2', 'jkandroid','yzand1','jkmjand_1','jkandjqb','jkandxedk','jkandpphd','jkoriand2']
jk_appid_ios = [ 'jkios','jkiosspeed', 'jkiossolar','jkioslunar','jkmjios_1','jkiosxjjsd','jkiosjqb','jkiosxysjd','jkiosjqd','jkiosxejsd'
                ,'jkiossjdjk','jkiosxjjk','jkiospphd','jkiosjyq','jkiosxedk','jkoriios']

# ACQ02

In [2]:
# label对应target，将0变换成1，1变换成0，label1对应target1，同样操作
# 读取申请时间(即这里的最后决策时间)
label = pd.read_csv('./data_raw/APP_5K_20171120_ADD_UGID.csv')
# label = label.drop(['month','user_gid','last_decision_dt','SamplingWeight'],axis=1).rename(columns={'create_time':'last_decision_tm'})
label = label[['ugid','create_time']].rename(columns={'create_time':'last_decision_tm'})

In [17]:
label.head()

,ugid,last_decision_tm
0,2ad5285a-c48b-466a-ab4b-591f41f10521,20AUG2017:00:07:22.967
1,3a89d1b3-07ca-4f90-b248-67cd19114949,12AUG2017:11:30:05.710
2,a8311a80-b7f8-4e2f-ae7e-3a3c4c1d8da4,04AUG2017:19:10:04.553
3,a3fa04ae-4665-4360-a62e-0faf5dc2e6eb,22AUG2017:18:52:00.110
4,b3e8d465-3de5-4564-9810-71f34be2b6ae,07AUG2017:00:21:11.143


In [3]:
# 读取原始数据

acq02 = pd.read_csv("./data_mid/20171122_data_acq02.201706_201708.csv", encoding = 'utf-8', error_bad_lines=False,
                    dtype = {'endTime':np.str},escapechar = '\\')

acq02.columns = ['ugid', 'zuid', 'appid', 'utime', 'ctime', 'ch', 'eleId', 'path', 'remote_addr', 'sessionid', 
                 'startTime', 'endTime', 'stayTime', 'val']

In [21]:
# 拼上首次申请日期
data = pd.merge(acq02, label, how='left', on = 'ugid')
# data = data[data.target == data.target]
data.shape

(31062, 15)

In [23]:
#为什么drop掉这么多字段？因为文件过大，drop掉后易于操作，另外drop掉的字段并不衍生特征
data = data.drop(['remote_addr', 'startTime','endTime'], axis=1)

acq02['country'] = acq02['location'].apply(lambda x : x.split('\t')[0])
acq02['province'] = acq02['location'].apply(lambda x : x.split('\t')[1])
acq02['city'] = acq02['location'].apply(lambda x : x.split('\t')[2])

In [24]:
val = '::1, 12:1:0, 250:12:0, :12:2'

In [25]:
def mergeFtr(ftr, temp):
    ftr = pd.merge(ftr, temp, how='left', on = 'ugid')
    return ftr

In [26]:
def splitVal(x):
    # !!!!拿val值两个冒号之间的值
    sp = []
    temp = re.split(':[012](?:,|$)', x)
    for item in temp:
        sp.append(item.split(':')[-1])
    return sp[:-1]

In [27]:
def getValProperty(x):
    # !!!!取所有第二个冒号之后的值
    return re.findall(':((?:0|1|2))(?:,|$)',x)

def inputBehavior(x):
    '''
    extract the character each time when user input
    '''
    temp = splitVal(x)
    input_character = '-'.join(temp)
    return input_character

def inputLength(x):
    """
    check the length of each input behavior
    """
    temp = splitVal(x)
    length = '0'
    for item in temp:
        if item!='':
            length += ','+str(len(item))
        else:
            length += ',0'
    return length

In [28]:
data['val'] = data['val'].replace('null', np.nan)

In [29]:
%%time

# 判断val值记录的长度,即活动的次数
data['input_len'] = data['val'].apply(lambda x : len(splitVal(x)) if x==x else 0).astype('int')

# 判断失去焦点的次数（查上报数据里面有多少个末位为2的记录）
data['lose_cnt'] = data['val'].apply(lambda x : getValProperty(x).count('2') if x==x else 0).astype('int')

# 判断获得焦点的次数（查上报数据里面有多少个末位为1的记录）
data['get_cnt'] = data['val'].apply(lambda x : getValProperty(x).count('1') if x==x else 0).astype('int')

CPU times: user 880 ms, sys: 0 ns, total: 880 ms
Wall time: 878 ms


In [30]:
%%time

# 计算所有上报记录中第一条记录的末位数
data['first'] = data['val'].apply(lambda x : getValProperty(x)[0] if x==x else 0).astype('str')

# 计算所有上报记录中最后一条记录的末位数
data['last'] = data['val'].apply(lambda x : getValProperty(x)[-1] if x==x else 0).astype('str')

# 提取所有两个冒号之间的值，会用-号链接
data['input_character']  = data['val'].apply(lambda x : inputBehavior(x) if x==x else 'None').astype('str')

# 提取所有两个冒号之间的值的长度
data['input_length'] = data['val'].apply(lambda x : inputLength(x) if x==x else 'None').astype('str')

CPU times: user 1.34 s, sys: 0 ns, total: 1.34 s
Wall time: 1.34 s


In [31]:
%%time

# 判断是否超过一次失去焦点
data['lose_m1'] = data['lose_cnt'].apply(lambda x : 1 if (x>1) and (x==x) else 0).astype('int')

# 判断是否超过一次获得焦点：1、如果首位是获得焦点，说明数据上报没有缺失，那么正常获得焦点次数为1，大于1为非正常值
                     # 2、如果首位不是获得焦点，说明数据上报有缺失，那么正常获得焦点次数为0，大于0为异常值
data['get_m1'] = data.apply(lambda x : 1 if ((x['get_cnt']>1) and (x['first']=='1')) or 
                                            ((x['get_cnt']>0) and (x['first']!='1')) else 0, axis=1).astype('int')

CPU times: user 1.3 s, sys: 0 ns, total: 1.3 s
Wall time: 1.29 s


In [32]:
def checkCP(x):
    # 提取所有的字符输入长度，转成list便于迭代
    x = [int(y) for y in x.split(',')]
    is_cp = ''
    for n in range(len(x)):
        if n==0:
            is_cp += '0'
        # 如果不是首位输入，那么输入的字符长度比上一次输入大超过2判断当次输入为复制粘贴
        else:
            if x[n] - x[n-1] >= 3:
                is_cp += '1'
            else:
                is_cp += '0'
    return is_cp

def checkChanged(x):
    # 判断是否有修改， 输入是每次输入字符长度input_length
    
    x = [int(y) for y in x.split(',')]
    x = list(filter(lambda y : y!=0, x))
    chng = ''
    for n in range(len(x)):
        if n>0:
            # 当次输入长度比上次一记录短1的话，判断当次输入为修改
            if x[n]-x[n-1]==-1:
                chng+='1'
            else:
                chng+='0'
    return chng

In [33]:
def extractFinal(x):
    x = x.split('-')
    return x[-1]

In [34]:
def checkFloat(x):
    a = len(re.findall('[0-9]',  str(x)))
    if a>0:
        return True
    else:
        return False

import re
def checkEnglish(x):
    try:
        a = len(re.findall('[a-z]',  x))
        if a>0:
            return True
        else:
            return False
    except:
        return True

In [35]:
#一些合法性的检查逻辑
#银行卡长度为16-19位
legal_input = {
     '_mobilenum' : [11],
     '_name' : [2,3,4],
     '_id' : [15,18],
     '_cardnum' : [x for x in range(16,20,1)]
}

In [36]:
def checkIllegal(x):
    
    # 对于非姓名类的输入，只要输入长度不在我的逻辑内就判断为不合法，1不合法，0合法 
    if 'name' not in x['button']:
        key = [m for m in legal_input.keys() if m in x['button']]
        if len(key) == 0:#不在我排查的范围内，则认为合法
            return 0
        else:
            inputlen = legal_input[key[0]]
            # x['lastInputLen']是输入值的长度，inputlen逻辑判断为合法的长度
            if  x['lastInputLen'] not in inputlen:
                return 1
            else:
                return 0
            
    # 对于姓名类的，不止检查输入长度，还检查是否存在英文字母和数字，如果存在任一，则判断为不合法     
    else:
        key = [m for m in legal_input.keys() if m in x['button']]
        inputlen = legal_input[key[0]]
        if (x['is_english'] == True)|(x['is_float'] == True):
            return 1 # 存在英文或数字任一，都返回1表示不合法
        elif (x['is_english'] == False)&(x['is_float'] == False)&(x['lastInputLen'] not in inputlen):
            return 1 # 当不存在英文和数字的时候，如果长度不对，也判断为不合法
        else:
            return 0 # 剩下的情况都判断为合法

In [37]:
%%time

# 判断是不是一条记录里没有输入，通过输入字符的长度判断
data['is_all_0'] = data.apply(lambda x : 1 if x['input_length'].count('0')-1 == x['input_len'] else 0, axis=1)

# 判断每次一输入是否为复制粘贴
data['is_cp'] = data.apply(lambda x : checkCP(x['input_length']) if (x['is_all_0'] == 0) and (x['input_length']!='None') else 'None', axis=1)

CPU times: user 2.51 s, sys: 0 ns, total: 2.51 s
Wall time: 2.51 s


In [38]:
%%time

# !!!!判断档次对话框输入中是否存在复制粘贴：is_cp字段中包含字符1
data['is_copy_paste'] = data.apply(lambda x : 1 if ((x['input_length'].count('1,') == 0) or 
                                                      (x['is_cp'].count('1')!=0)) 
                                     and (x['is_all_0'] == 0) else 0, axis=1)

CPU times: user 1.03 s, sys: 0 ns, total: 1.03 s
Wall time: 1.03 s


In [39]:
%%time

# 判断每次输入是否有修改，返回一个逗号分隔的判断字符（'000110'表示在第四次第五次输入修改）
data['change'] = data['input_length'].apply(lambda x : checkChanged(x) if( x==x) and (x!='None') else 'None')

# 判断当次对话框输入是否存在修改，返回修改次数，对于例子来说返回2
data['change_cnt'] = data['change'].apply(lambda x : x.count('1') if (x!='') and (x!='None') else 0).astype(int)

CPU times: user 424 ms, sys: 0 ns, total: 424 ms
Wall time: 424 ms


In [40]:
%%time

# 提取输入完成时的输入框中的字符
data['lastInput'] = data['input_character'].apply(lambda x : extractFinal(x) if( x==x) and (x!='None') else 'None')

# 提取输入完成时输入框中的字符的长度
data['lastInputLen'] = data['lastInput'].apply(lambda x : len(x.replace(' ','')) if ( x==x) and (x!='None') else 0).astype(int)

CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 105 ms


In [41]:
%%time
# 判断最后一位输入里是否存在数字，有任何一个数字就返回True，否则返回False
data['is_float'] = data['lastInput'].apply(lambda x : checkFloat(x) if x==x else False)

# 判断最后一位输入里是否存在英文字母，有任何一个字母就返回True，否则返回False
data['is_english'] = data['lastInput'].apply(lambda x : checkEnglish(x) if x==x else True)

CPU times: user 204 ms, sys: 0 ns, total: 204 ms
Wall time: 204 ms


In [42]:
# 定义核心事件

path = {'/card/certification.html': 'certification',
        '/card/addcreditcard.html': 'addcreditcard',
        '/login.html': 'login',
        '/regisiter.html': 'register',
        '/card/addcard.html': 'addcard',
        '/lend/lendInfo.html': 'lendinfo',
        '/lend/lendbycard.html': 'lendinfo',
        '/certification.html': 'certification'}


In [43]:
# 定义核心输入框

button = {'jk_aci_01_01' : 'register_mobilenum',
          'jk_aci_01_02' : 'register_dynamic_code',
          'jk_aci_01_03' : 'register_invitation',
          'jk_aci_02_01' : 'certification_name',
          'jk_aci_02_02' : 'certification_id',
          'jk_aci_02_03' : 'certification_cardnum',
          'jk_aci_02_05' : 'certification_mobilenum',
          'jk_aci_02_06' : 'certification_dynamic_code',
          'jk_aci_03_01' : 'addcard_name',
          'jk_aci_03_02' : 'addcard_cardnum',
          'jk_aci_03_04' : 'addcard_mobilenum',
          'jk_aci_04_01' : 'addcreditcard_name',
          'jk_aci_04_02' : 'addcreditcard_cardnum',
          'jk_aci_04_04' : 'addcreditcard_mobilenum'}


In [44]:
# 提取输入框、和访问页面的title，不重复
path_events = list(set([y for x,y in path.items()]))
button_events = list(set([y for x,y in button.items()]))

In [45]:
%%time

#将path重命名为url,重新给path值为path的值，如果无值则为others
data = data.rename(columns = {'path':'url'})

# 如果url在核心事件的列表中，则path赋值为这个链接的title，如果不在就赋值为others
data['path'] = data['url'].apply(lambda x : path.get(x, 'others'))

# 如果url在核心输入框的列表中，则path赋值为这个链接的title，如果不在就赋值为others
data['button'] = data['eleId'].apply(lambda x : button.get(x, 'other_buttons'))

CPU times: user 68 ms, sys: 40 ms, total: 108 ms
Wall time: 107 ms


In [46]:
data.head()

,ugid,zuid,appid,utime,ctime,ch,eleId,url,sessionid,stayTime,...,is_cp,is_copy_paste,change,change_cnt,lastInput,lastInputLen,is_float,is_english,path,button
0,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1.497178e+09,NaN,jk_aci_02_01,/card/certification.html,1497178289927,3853.0,...,000000,0,000,0,方兰香,3,False,False,certification,certification_name
1,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1.497178e+09,NaN,jk_aci_02_02,/card/certification.html,1497178289927,15454.0,...,000000000000000000000,0,000000000000000000,0,152631197209250064,18,True,False,certification,certification_id
2,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1.497178e+09,NaN,jk_aci_02_03,/card/certification.html,1497178289927,15498.0,...,0000000000000000000000,0,0000000000000000000,0,6228 4830 6812 0617 174,19,True,False,certification,certification_cardnum
3,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1.497178e+09,NaN,jk_aci_02_05,/card/certification.html,1497178289927,4522.0,...,00000000000000,0,00000000000,0,18347428919,11,True,False,certification,certification_mobilenum
4,1337df2a-7323-4d64-99a4-f2182c315c0d,89409acd6a5c31638d174620bceee83e|123456789002,jkandroid,NaN,1.497178e+09,NaN,jk_aci_02_06,/card/certification.html,1497178289927,2881.0,...,0000000,0,0000,0,7200,4,True,False,certification,certification_dynamic_code


In [47]:
%%time

# 判断是否不合法
data['isIllegal'] = data.apply(lambda x : checkIllegal(x),axis=1).astype(int)

CPU times: user 2.08 s, sys: 112 ms, total: 2.19 s
Wall time: 2.19 s


In [48]:
%%time

# 清理申请时间格式
data['last_decision_tm'] = data['last_decision_tm'].replace('DEC', '-12-',regex=True).\
replace('NOV', '-11-',regex=True).\
replace('OCT', '-10-',regex=True).\
replace('SEP', '-09-',regex=True).\
replace('AUG', '-08-',regex=True).\
replace('JUL', '-07-',regex=True).\
replace('JUN', '-06-',regex=True).\
replace('MAY', '-05-',regex=True).\
replace('APR', '-04-',regex=True).\
replace('MAR', '-03-',regex=True).\
replace('FEB', '-02-',regex=True).\
replace('JAN', '-01-',regex=True)

CPU times: user 280 ms, sys: 0 ns, total: 280 ms
Wall time: 280 ms


In [49]:
%%time
# 清理ctime，如果ctime没有，用utime代替，如果全部没有就不管
data['ctime'] = data.apply(lambda x : x['utime'] if x['ctime'] == 'null' else x['ctime'], axis=1)

# 转换申请时间格式
data['last_decision_tm'] = data['last_decision_tm'].apply(lambda x :datetime.strptime(x, '%d-%m-%Y:%H:%M:%S.%f'))

CPU times: user 1.64 s, sys: 108 ms, total: 1.75 s
Wall time: 1.75 s


In [50]:
# ctime格式清理
data.ctime = data.ctime.astype('float')

In [51]:
%%time
# 把ctime时间戳转换为标准时间格式
data['ctime_stamp'] = pd.to_datetime(data.ctime, unit='s')
#data['last_decision_tm'] = pd.to_datetime(data['last_decision_tm'])

CPU times: user 312 ms, sys: 4 ms, total: 316 ms
Wall time: 314 ms


In [52]:
%%time
# 把以毫秒为单位的停留时间转换为以秒为单位的停留时间
data['stayTime'] = data['stayTime'].replace('null',np.nan).astype(float) / 1000.0

CPU times: user 44 ms, sys: 108 ms, total: 152 ms
Wall time: 29.5 ms


In [ ]:
# %%time
# 存储中间文件
# data.to_csv("./data_mid/20171016_data_acq02_interim.201611_201705.txt", index=False, encoding = 'utf-8')

## features ectractors

In [53]:
# !!!!转时区，为什么要转
data['ctime_stamp_new'] = pd.to_datetime(data.ctime+to_s, unit='s')

In [54]:
data = data[data.last_decision_tm == data.last_decision_tm]

In [55]:
from datetime import timedelta
def getWindowedData(data, apply_time, data_time, timewindow):
    '''
    获取统计时间窗口数据
    '''
    p = datetime.now()
    data = data[((data[data_time]+timedelta(timewindow)) > data[apply_time])&(data[data_time]<data[apply_time])]
    print('  - filtering data using %s' % (datetime.now()-p))
    return data

In [56]:
def groupbyMethod(data, groupby_col, val_col, method, rename = ''):
    '''
    groupby计算代码，data待计算的数据集，groupby_col是计算主键，val_col是需要计算的列名，method是计算逻辑，rename是是否需要改名
    '''
    temp = getattr(data.groupby(groupby_col)[val_col], method)().reset_index()
    if rename:
        temp = temp.rename(columns = {val_col : rename})
    return temp

In [57]:
def initializeFeatMapGeneral(groupby = ['ugid'], time_window = '1m',name_sub = ''):
    prefix = 'last'+time_window+'_'+name_sub+'_' if name_sub!='' else 'last'+time_window+'_'
    '''
    依次为客户级1.记录数，2.复制粘贴记录数，3.总更改次数，4.平均更改次数，5.总停留时间，6.平均停留时间，7.获取焦点大于1的总记录数
    8.失去焦点大于1的总记录数，9.最终输入不合法的总次数
    '''
    # map有三列，第一列是输出特征的含义，第二列是计算特征时用到的列名， 第三列是计算用到的操作
    feat_map_general = [['input_cnt_agg','val', 'count'],
                ['input_cp_cnt_agg', 'is_copy_paste', 'sum'],
                ['input_change_cnt_agg', 'change_cnt', 'sum'],
                ['input_change_cnt_avg', 'change_cnt', 'mean'],
                ['input_staytime_agg', 'stayTime', 'sum'],
                ['input_staytime_avg', 'stayTime', 'mean'],
                ['input_get_focus_m1_cnt_agg', 'get_m1', 'sum'],
                ['input_lose_focus_m1_cnt_agg', 'lose_m1', 'sum'],
                ['input_illegal_cnt_agg', 'isIllegal', 'sum']]
    feat_map_general = [(prefix+x,groupby,y,z) for x,y,z in feat_map_general]
    return feat_map_general

In [58]:
def initializeFeatMapAdd(groupby=['ugid'], time_window = '1m',name_sub = ''):
    '''
    补充额外特征1.输出最大变更次数，2.最大停留时间
    '''
    assert time_window in ['1m', '1w', '1d'], 'check time window'
    prefix = 'last'+time_window+'_'+name_sub+'_' if name_sub!='' else 'last'+time_window+'_'
    feat_map_add = [
                ['input_change_cnt_max', 'change_cnt', 'max'],
                ['input_staytime_max', 'stayTime', 'max']]
    feat_map_add = [(prefix+x,groupby,y,z) for x,y,z in feat_map_add]

    return feat_map_add

In [59]:
def featureExtractorPerc(data, time_window = '1m', name_sub = ''):
    '''
    算某些特征的百分比，通过配置feat进行
    依次为1.复制粘贴记录数占比，2.输入不合法占比，3.获取焦点大于1占比，4。失去焦点大于1占比
    '''
    
    # 对feat的第一列求百分比，结果命名为第二列
    feat = [('input_cp_cnt_agg', 'input_cp_perc'), 
            ('input_illegal_cnt_agg', 'input_illegal_perc'), 
            ('input_get_focus_m1_cnt_agg', 'input_get_focus_m1_perc'),
            ('input_lose_focus_m1_cnt_agg', 'input_lose_focus_m1_perc')]
    if name_sub == '':
        agg_col = 'last'+time_window+'_'+'input_cnt_agg'
        feat = [('last'+time_window+'_'+x,'last'+time_window+'_'+y) for x,y in feat]
    else:
        agg_col = 'last'+time_window+'_'+name_sub+'_input_cnt_agg'
        feat = [('last'+time_window+'_'+name_sub+'_'+x,
                 'last'+time_window+'_'+name_sub+'_'+y) for x,y in feat]
    for sub_col, new in feat:
        
        data[new] = data[sub_col] / data[agg_col].astype('float')
    return data

In [60]:
# 特征提取代码
def extract(data, apply_time, data_time, groupby = ['ugid'],
            time_window = '1m', name_sub = '',additional = False):
    '''
    输入：data时间窗口后的原始数据，apply_time是申请时间的列名，data_time是数据采集时间，groupby是主键列表，
    time_window就是时间窗口筛选逻辑，name_sub是前缀，additional是额外特征
    '''
    p = datetime.now()
    # 提取全量数据中unique ugid
    ftr = pd.DataFrame(list(set(data['ugid'].values)), columns = ['ugid'])
    
    # 提取特征配置文件
    # fm是基础特征
    fm = initializeFeatMapGeneral(groupby, time_window = time_window, name_sub = name_sub)
    if additional:
        # 补充额外特征
        fm.extend(initializeFeatMapAdd(groupby, time_window = time_window,name_sub = name_sub))
    total_len = len(fm)
    
    # 根据map去计算特征
    for n,item in enumerate(fm):
        temp = groupbyMethod(data, item[1], item[2], item[3], item[0]).fillna(-1)
        ftr = mergeFtr(ftr, temp).fillna(-1)
        if float(n) == np.ceil(total_len/2):
            print('      - feature extracting using %s' %(datetime.now()-p))
    ftr = featureExtractorPerc(ftr, time_window = time_window, name_sub = name_sub)
    return ftr

In [61]:
# 特征提取的主入口
def featureExtractor(data, apply_time, data_time, groupby = ['ugid'],
                     time_window = '1m', name_sub = '',additional = False):
    # 定义一个时间-前缀的配置
    tw = {'1m':30, '1w':7, '1d':1}
    assert time_window in tw, 'check time window' # 逻辑检查
    
    # 筛选时间窗口
    data = getWindowedData(data, apply_time, data_time, tw.get(time_window))
    
    # 返回特征提取
    return extract(data, apply_time, data_time, groupby = ['ugid'],
            time_window = time_window, name_sub = name_sub,additional = additional)

In [62]:
# session2、3的主入口
def featureExtractorID(data,apply_time, data_time,  groupby = ['ugid'],
                     time_window = '1m', name_sub = '',additional = False, group_list = path_events, 
                     group_col = 'path'):
    '''
    group list就是核心事件、核心按钮的list
    '''
    tw = {'1m':30, '1w':7, '1d':1}
    assert time_window in tw, 'check time window'
    data = getWindowedData(data, apply_time, data_time, tw.get(time_window))
    ftr = pd.DataFrame(list(set(data['ugid'].values)), columns = ['ugid'])
    for name in group_list:
        print('  - %s' %name)
        temp = data[data[group_col] == name]
        ftr = mergeFtr(ftr, extract(temp, apply_time, data_time, groupby = ['ugid'],
                                    time_window = time_window, name_sub = name,additional = additional))
    return ftr

In [63]:
print (data)

                                       ugid  \
0      1337df2a-7323-4d64-99a4-f2182c315c0d   
1      1337df2a-7323-4d64-99a4-f2182c315c0d   
2      1337df2a-7323-4d64-99a4-f2182c315c0d   
3      1337df2a-7323-4d64-99a4-f2182c315c0d   
4      1337df2a-7323-4d64-99a4-f2182c315c0d   
5      670a5ec7-2940-404c-bc4c-09b0d73175f2   
6      670a5ec7-2940-404c-bc4c-09b0d73175f2   
7      670a5ec7-2940-404c-bc4c-09b0d73175f2   
8      670a5ec7-2940-404c-bc4c-09b0d73175f2   
9      670a5ec7-2940-404c-bc4c-09b0d73175f2   
10     21426397-8707-46c4-b89f-5446848c50ed   
11     21426397-8707-46c4-b89f-5446848c50ed   
12     21426397-8707-46c4-b89f-5446848c50ed   
13     e9776d5f-61a2-4116-833c-1d0945abc673   
14     e9776d5f-61a2-4116-833c-1d0945abc673   
15     e9776d5f-61a2-4116-833c-1d0945abc673   
16     e9776d5f-61a2-4116-833c-1d0945abc673   
17     e9776d5f-61a2-4116-833c-1d0945abc673   
18     e9776d5f-61a2-4116-833c-1d0945abc673   
19     e9776d5f-61a2-4116-833c-1d0945abc673   
20     e9776d

# 整个acq02表衡量输入行为

## 叠加一个时间窗口（过去一天、一个星期、一个月）

### 基础的输入统计（无视在哪个页面输入、在哪个对话框输入）

### 页面输入统计（统计核心事件的输入情况）

### 对话框输入统计（统计核心对话框的输入情况）

层级越来越细

In [64]:
%%time
# 特征提取的主入口

# last1d 提取无视核心事件、核心输入框的统计
print('feature_extracting - last one day, session 1')
ftr_1d = featureExtractor(data,'last_decision_tm', 'ctime_stamp_new', time_window = '1d', additional = True)

# last1d 提取无视输入框，只统计页面行为
print('feature_extracting - last one day, session 2')
ftr_1d = mergeFtr(ftr_1d, featureExtractorID(data,'last_decision_tm', 'ctime_stamp_new', time_window = '1d', additional = True,
                              group_list = path_events, group_col = 'path'))

# last1d 具体到输入框的行为统计
print('feature_extracting - last one day, session 3')
ftr_1d =  mergeFtr(ftr_1d, featureExtractorID(data,'last_decision_tm', 'ctime_stamp_new', time_window = '1d', additional = True,
                              group_list = button_events, group_col = 'button'))

feature_extracting - last one day, session 1
  - filtering data using 0:00:00.022629
      - feature extracting using 0:00:00.101291
feature_extracting - last one day, session 2
  - filtering data using 0:00:00.012149
  - addcreditcard
      - feature extracting using 0:00:00.050761
  - register
      - feature extracting using 0:00:00.062732
  - lendinfo
      - feature extracting using 0:00:00.043590
  - addcard
      - feature extracting using 0:00:00.039283
  - login
      - feature extracting using 0:00:00.038978
  - certification
      - feature extracting using 0:00:00.075730
feature_extracting - last one day, session 3
  - filtering data using 0:00:00.019932
  - addcard_name
      - feature extracting using 0:00:00.046410
  - certification_mobilenum
      - feature extracting using 0:00:00.061235
  - register_invitation
      - feature extracting using 0:00:00.034306
  - certification_name
      - feature extracting using 0:00:00.040084
  - addcreditcard_cardnum
      - feature

In [65]:
%%time

# last1w
print('feature_extracting - last one week, session 1')
ftr_1w = featureExtractor(data,'last_decision_tm', 'ctime_stamp_new', time_window = '1w', additional = True)
print('feature_extracting - last one week, session 2')
ftr_1w = mergeFtr(ftr_1w, featureExtractorID(data,'last_decision_tm', 'ctime_stamp_new', time_window = '1w', additional = True,
                              group_list = path_events, group_col = 'path'))
print('feature_extracting - last one week, session 3')
ftr_1w =  mergeFtr(ftr_1w, featureExtractorID(data,'last_decision_tm', 'ctime_stamp_new', time_window = '1w', additional = True,
                              group_list = button_events, group_col = 'button'))

feature_extracting - last one week, session 1
  - filtering data using 0:00:00.020670
      - feature extracting using 0:00:00.105207
feature_extracting - last one week, session 2
  - filtering data using 0:00:00.012118
  - addcreditcard
      - feature extracting using 0:00:00.051209
  - register
      - feature extracting using 0:00:00.045573
  - lendinfo
      - feature extracting using 0:00:00.043162
  - addcard
      - feature extracting using 0:00:00.039262
  - login
      - feature extracting using 0:00:00.039355
  - certification
      - feature extracting using 0:00:00.078977
feature_extracting - last one week, session 3
  - filtering data using 0:00:00.011301
  - addcard_name
      - feature extracting using 0:00:00.031053
  - certification_mobilenum
      - feature extracting using 0:00:00.039090
  - register_invitation
      - feature extracting using 0:00:00.030744
  - certification_name
      - feature extracting using 0:00:00.037737
  - addcreditcard_cardnum
      - feat

In [66]:
%%time

# last1m
print('feature_extracting - last one month, session 1')
ftr_1m = featureExtractor(data,'last_decision_tm', 'ctime_stamp_new', time_window = '1m', additional = True)
print('feature_extracting - last one month, session 2')
ftr_1m = mergeFtr(ftr_1m, featureExtractorID(data,'last_decision_tm', 'ctime_stamp_new', time_window = '1m', additional = True,
                              group_list = path_events, group_col = 'path'))
print('feature_extracting - last one month, session 3')
ftr_1m =  mergeFtr(ftr_1m, featureExtractorID(data,'last_decision_tm', 'ctime_stamp_new', time_window = '1m', additional = True,
                              group_list = button_events, group_col = 'button'))


feature_extracting - last one month, session 1
  - filtering data using 0:00:00.021107
      - feature extracting using 0:00:00.110837
feature_extracting - last one month, session 2
  - filtering data using 0:00:00.021267
  - addcreditcard
      - feature extracting using 0:00:00.060695
  - register
      - feature extracting using 0:00:00.050210
  - lendinfo
      - feature extracting using 0:00:00.049282
  - addcard
      - feature extracting using 0:00:00.039964
  - login
      - feature extracting using 0:00:00.043191
  - certification
      - feature extracting using 0:00:00.091668
feature_extracting - last one month, session 3
  - filtering data using 0:00:00.016537
  - addcard_name
      - feature extracting using 0:00:00.038491
  - certification_mobilenum
      - feature extracting using 0:00:00.048052
  - register_invitation
      - feature extracting using 0:00:00.038400
  - certification_name
      - feature extracting using 0:00:00.046858
  - addcreditcard_cardnum
      - f

In [67]:
#ftr = mergeFtr(ftr_1d, ftr_1m)
ftr = mergeFtr(ftr_1m, ftr_1d)

In [68]:
ftr = mergeFtr(ftr, ftr_1w)

In [69]:
ftr.shape

(2946, 946)

In [70]:
%%time
ftr.to_csv('./data_last/20171123_feature_acq02_201706_201708.csv', index=False)

CPU times: user 2.52 s, sys: 84 ms, total: 2.61 s
Wall time: 2.71 s


In [71]:
ftr

,ugid,last1m_input_cnt_agg,last1m_input_cp_cnt_agg,last1m_input_change_cnt_agg,last1m_input_change_cnt_avg,last1m_input_staytime_agg,last1m_input_staytime_avg,last1m_input_get_focus_m1_cnt_agg,last1m_input_lose_focus_m1_cnt_agg,last1m_input_illegal_cnt_agg,...,last1w_addcard_mobilenum_input_staytime_avg,last1w_addcard_mobilenum_input_get_focus_m1_cnt_agg,last1w_addcard_mobilenum_input_lose_focus_m1_cnt_agg,last1w_addcard_mobilenum_input_illegal_cnt_agg,last1w_addcard_mobilenum_input_change_cnt_max,last1w_addcard_mobilenum_input_staytime_max,last1w_addcard_mobilenum_input_cp_perc,last1w_addcard_mobilenum_input_illegal_perc,last1w_addcard_mobilenum_input_get_focus_m1_perc,last1w_addcard_mobilenum_input_lose_focus_m1_perc
0,b8ab4e12-639f-4f89-9440-6a28008784e5,30,4,17,0.566667,225.963,8.070107,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,de2cd5a1-2aaf-485f-a6ae-b527bfd18737,12,1,9,0.750000,198.365,16.530417,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27128422-47a1-44fa-9cfb-aaebd1ce4773,10,1,0,0.000000,117.838,11.783800,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,70f2d845-9eae-498f-a37a-b82f0d3a5672,5,0,0,0.000000,36.730,7.346000,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,53209e6c-181d-4e45-9c9d-33cf2554d5c1,9,1,3,0.333333,78.693,8.743667,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,075579b6-4358-4d0a-8345-cd2656bb7202,5,0,0,0.000000,38.998,7.799600,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,c7e6a6e2-0c49-4f03-8da1-f8e93f492a10,14,2,19,1.357143,288.458,20.604143,2,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5c57e91e-84b3-4645-926b-aecfa4c64ded,11,4,1,0.090909,51.001,4.636455,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,71ecf820-2c92-4675-b427-7e4e0da1ef06,7,0,9,1.285714,94.660,13.522857,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,d20be0b7-e906-4cf9-aa69-e5d94d3e586c,7,2,1,0.142857,43.822,6.260286,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
